In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors


# Recipe recommendation

In my mind, a powerful approach to recommending recipes would be if I could say something like "of all people who liked recipes A and B, 75% also liked recipe C.  You like A and B, so you should try C too."  This would allow the discovery of other quality recipes, not necessarily similar to one I already know, but enjoyed by people with a similar taste.  However, I can't do this when I'm the only person I have data on.

Instead, I must rely on features of the recipes themselves.  Since I did not scrape ratings, time required, or other meta-data, I have only the ingredients to go off of.  So, for a first pass at recommending recipes, I'll take a given "loved" recipe and find the k nearest recipes to it in the database.  There are drawbacks to this approach, namely a recipe with similar ingredients will not necessarily be as good, and moreover, if it's too similar, I'd probably rather stick with the recipe I already know.  The point is to discover new foods I wouldn't find otherwise.

However, the advantage of this approach is it's really easy to do.



In [6]:
df = pd.read_csv('one_hot_data.csv')

There's no train-test split here.  However, I do want to shuffle the data and a 100-0 split is an easy way to do that.

In [7]:
train, test = train_test_split(df, test_size=0)

In [8]:
names = train['Recipe_name']
data = train.drop(columns=['Recipe_name', 'Rating'])


In [9]:
N=6
nbrs = NearestNeighbors(n_neighbors=N, algorithm='ball_tree').fit(data)

In [12]:
def recommend(recipe):
    point = df[df['Recipe_name']==recipe ].drop(columns=['Recipe_name', 'Rating'])
    dists, inds = nbrs.kneighbors(point)
    for i, d in zip(inds[0][1:], dists[0][1:]): # ignore the first which is just the searched recipe
        print('Distance {:.4f}:  {}'.format(d, names.iloc[i]))

## TESTS

Note, if this gets re-run, the data may be in a different order, which will affect the outcome of the nearest neighbors when there's a tie.  So in my comments after each test, I refer to recipes that you may or may not see in the outcome of the test.

### Test 1: Mac and Cheese
We first test by finding the five nearest recipes to mac and cheese.  I've never made this recipe, but it seems a fairly unique dish so I'm curious to see what turns up.

In [13]:
recommend('Adult Mac and Cheese')

Distance 1.0000:  Fettuccine Alfredo
Distance 1.4142:  Pasta With Brown Butter Whole Lemon and Parmesan
Distance 1.7321:  Cacio e Pepe
Distance 1.7321:  Pasta al Limone
Distance 2.0000:  Make-Ahead Mashed Potatoes


Most of these recipes are fairly similar to the search point, simple pastas with cream and/or cheese.

### Test 2: Coconut corn and grains
Coconut-Creamed Corn and Grains is another very unique dish.
I actually really liked this one when we made it.

In [14]:
recommend('Coconut-Creamed Corn and Grains')

Distance 3.4641:  Orange Fish in Parchment
Distance 3.4641:  Aromatic Wilted Greens with Coconut Milk
Distance 3.4641:  Vegan Creamed Spinach
Distance 3.4641:  Vegan Butternut Squash Soup
Distance 3.6056:  Coconut Rice Noodles with Ginger and Turmeric


Looks like nothing is really nearby this recipe.  

A result of this simple method is that two recipes will be deemed similar if, as in this example, they both have ginger, coconut, cilantro, and garlic; while one is a squash-based soup and the other is a grain-based curry type thing.  This is a drawback if I truly want similar recipes; but maybe it's a plus if I want to find new recipes with flavors I know I like.


The fried brown rice looks more similar to the candidate recipe, and actually looks really good, maybe I'll make that one.

### Test 3: Cauliflower Bolognese

I love cauliflower and I love pasta and I love this recipe.  Let's see what's nearby.

In [15]:
recommend('Cauliflower Bolognese')

Distance 2.6458:  Brothy Pasta with Chickpeas
Distance 3.0000:  Cauliflower Steaks with Olive Relish and Tomato Sauce
Distance 3.0000:  Red Pesto Pasta
Distance 3.0000:  Pasta al Pomodoro
Distance 3.0000:  Bucatini with Butter-Roasted Tomato Sauce


The first result (Brothy Pasta with Chickpeas) is a recipe I have made several times, I like it a lot.  Seafood seems like a good leap away from the query recipe.

In [16]:
recommend('Fish Tacos al Pastor')

Distance 2.4495:  Pork Volcánes al Pastor
Distance 2.8284:  Tacos al Pastor
Distance 3.1623:  Orange Fish in Parchment
Distance 3.1623:  Citrus-Marinated Chicken Thighs
Distance 3.1623:  Grilled Fish Tacos


In [17]:
recommend('Digaag Qumbe (Yogurt-Coconut Chicken)')

Distance 3.3166:  Vegan Coconut Lentil Soup
Distance 3.4641:  Chicken Tikka Masala
Distance 3.6056:  Spiced Chicken with Chickpeas and Spinach
Distance 3.6056:  Coconut-Curry Braised Chicken Thighs
Distance 3.6056:  Slow-Cooker Indian Spiced Chicken with Tomato and Cream


In [18]:
recommend('Herby Barley Salad with Butter-Basted Mushrooms')

Distance 2.4495:  Grilled Chanterelle Mushrooms with Parsley and Lemon
Distance 2.6458:  Creamy Pasta with Crispy Mushrooms
Distance 2.8284:  No-Fail Roast Chicken with Lemon and Garlic
Distance 2.8284:  Garlic Broth
Distance 2.8284:  Chicken Under a Skillet with Lemon Pan Sauce
